## cNMF analysis - (using Decipher Seq v2)

### Following pipeline from DECIPHER-seq-v2 (in R) (as suggested by UCSF): https://github.com/dasuperville/DECIPHER-seq-v2 
### originally testing on DC2 only object (small object) to get code established, then can try on full atlas 

# Generate environment, clone repo/ install packages

``` bash
#generate new environment called decipher
#conda create -n decipher2 -c conda-forge \
  #r-base=4.4 \
  #r-essentials \
  #compilers \
  #make \
  #cmake \
  #pkg-config \
  #libblas \
  #liblapack \
  #libopenblas \
  #-y

conda create -n decipher -c conda-forge r-base=4.4 -y

conda activate decipher
#clone decipher-seqv2 repo (while in scratch)
git clone https://github.com/dasuperville/DECIPHER-seq-v2.git
cd DECIPHER-seq-v2
##initialize R
R
#install.packages("renv")  #install renv - project-specific package version manager
# ensure correct versions:
renv::restore()  #keep getting error with mvtnorm

library(Seurat)
library(ggplot2)

source('R/DECIPHER-seq.functions.R')
source('R/DECIPHER-seq.util.R')
```

In [1]:
version

               _                           
platform       x86_64-conda-linux-gnu      
arch           x86_64                      
os             linux-gnu                   
system         x86_64, linux-gnu           
status                                     
major          4                           
minor          5.2                         
year           2025                        
month          10                          
day            31                          
svn rev        88974                       
language       R                           
version.string R version 4.5.2 (2025-10-31)
nickname       [Not] Part in a Rumble      

In [ ]:
# load required packages/ scripts 
library(Seurat)
library(ggplot2)
source('R/DECIPHER-seq.functions.R')
source('R/DECIPHER-seq.util.R')

## install required packages
#### a lot of the required packages need r4.4, made new hpc env called envr44 which has r4.4, installing all packages into this, then need to work from this
#### missing packages:
* Seurat `mamba install -c conda-forge r-seurat` - done envnew-done
* sceasy in R `remotes::install_github("cellgeni/sceasy")` - done
* rliger in R `remotes::install_github("MacoskoLab/liger")` - done (first had to install a LOT of dependencies with mamba/conda and load modules)
* RANN `mamba install -c conda-forge r-rann` - done
* rlist `mamba install -c conda-forge r-rlist` - done
* ape `mamba install -c conda-forge r-ape` - done
* geiger `mamba install -c conda-forge r-geiger` - done
* wBoot `mamba install -c conda-forge r-wboot` - no longer available, installed boot instead 
* wTO in R `remotes::install_github("deisygysi/wTO")` (1st had to set git token) - done
* fgsea `remotes::install_github("ctlab/fgsea")` - done
* msigdbr `mamba install -c conda-forge r-msigdbr` - done
* rstatix `mamba install -c conda-forge r-rstatix` - done

In [26]:
library(rliger)

ERROR: Error in library(rliger): there is no package called ‘rliger’


In [24]:
library(sceasy)

In [ ]:
library(geiger)
library(igraph)
library(dplyr)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:igraph’:

    as_data_frame, groups, union


The following object is masked from ‘package:matrixStats’:

    count


The following object is masked from ‘package:ape’:

    where


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [ ]:
library(RANN)
library(Seurat)
library(reticulate)
library(ggplot2)
library(rlist)
library(ape)
library(matrixStats)
library(patchwork)

In [ ]:
install.packages("rliger")

In [3]:
#set wd
setwd('/scratch/user/s4436039/scdata/cNMF_Decipher')
getwd()

[1] "/scratch/user/s4436039/scdata/cNMF_Decipher"

In [4]:
library(Seurat)
library(sceasy)
library(reticulate)
library(ggplot2)

ERROR: Error in library(Seurat): there is no package called ‘Seurat’


In [4]:
h5ad_path = "/scratch/user/s4436039/scdata/Myeloid_Objects/20250629_DC2_bbknn_clustered2.h5ad"

sceasy::convertFormat(h5ad_path, from="anndata", to="seurat",
                       outFile='20250629_DC2_bbknn_clustered2_convert.rds')

X -> counts



An object of class Seurat 
2001 features across 16643 samples within 1 assay 
Active assay: RNA (2001 features, 0 variable features)
 2 layers present: counts, data
 2 dimensional reductions calculated: pca, umap

In [5]:
dc2.data = readRDS('20250629_DC2_bbknn_clustered2_convert.rds')

In [6]:
head(dc2.data@meta.data)

,nCount_RNA,nFeature_RNA,sample_type,cancer_type,patient_id,sample_id,percent.mt,site,sample_type_major,cancer_subtype,⋯,2025-DC2-true_08_sub5,2025-DC2-true_08_sub5-1,cDC2A_sig,cDC2B_sig,cDC2_A-B,cDC2_Atlas_Subclusters,DC_CD207_Moon_sig,cDC2A_Minutti_sig,DC2_CD207_Gui_sig,DC2_CD1A_Cheng_sig
,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
GSE215120_AM1_AAATGCCCAGAGCCAA-1,17143,2659,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.224990,skin,primary tumour,Acral Melanoma,⋯,0,0,0.1477201,0.5975502,cDC2B,1,-0.2851657,0.06103545,-0.22797638,-0.07507553
GSE215120_AM1_ACACTGATCCACTGGG-1,18333,2845,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.063656,skin,primary tumour,Acral Melanoma,⋯,0,0,-0.1008109,0.1814663,cDC2B,1,-0.2634986,-0.02531348,-0.22925684,-0.26723838
GSE215120_AM1_ACAGCCGCAAACCTAC-1,10581,2182,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.050846,skin,primary tumour,Acral Melanoma,⋯,0,0,-0.2218786,-0.6181285,cDC2B,1,-0.3915761,-0.22187855,-0.29580870,-0.39194336
GSE215120_AM1_ACGTCAATCCGCATCT-1,9833,2074,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,1.311909,skin,primary tumour,Acral Melanoma,⋯,2,2,-0.2279209,-0.1070891,cDC2B,5,-0.2408973,-0.22792086,-0.26391311,-0.37996750
GSE215120_AM1_ACTTGTTTCTGAAAGA-1,7887,1846,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.738684,skin,primary tumour,Acral Melanoma,⋯,2,2,-0.1990670,0.2832089,cDC2B,5,-0.1064612,-0.19906701,-0.20770551,-0.09538891
GSE215120_AM1_AGAGCTTGTACAGTTC-1,17339,2817,tumour,Acral Melanoma,AM1,GSE215120_Acral_MEL_AM1,2.612607,skin,primary tumour,Acral Melanoma,⋯,0,0,-0.2124498,0.6465073,cDC2B,1,-0.3503089,0.09326834,0.04285425,-0.22176579


In [12]:
install.packages("rliger")

also installing the dependencies ‘pROC’, ‘checkmate’, ‘easy.utils’, ‘sccore’, ‘hdf5r.Extra’, ‘HighFive’, ‘leidenAlg’, ‘RcppPlanc’


Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/src/contrib/pROC_1.19.0.1.tar.gz': Timeout of 60 seconds was reached”


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  cannot open URL 'https://cran.r-project.org/src/contrib/pROC_1.19.0.1.tar.gz'


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘pROC’ failed”
Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/src/contrib/checkmate_2.3.3.tar.gz': Timeout of 60 seconds was reached”


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  cannot open URL 'https://cran.r-project.org/src/contrib/checkmate_2.3.3.tar.gz'


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘checkmate’ failed”
Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/src/contrib/easy.utils_0.1.0.tar.gz': Timeout of 60 seconds was reached”


...

Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘leidenAlg’ failed”
Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/src/contrib/RcppPlanc_2.0.13.tar.gz': Timeout of 60 seconds was reached”


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  cannot open URL 'https://cran.r-project.org/src/contrib/RcppPlanc_2.0.13.tar.gz'


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘RcppPlanc’ failed”
Warning message in download.file(url, destfile, method, mode = "wb", ...):
“URL 'https://cran.r-project.org/src/contrib/rliger_2.2.1.tar.gz': Timeout of 60 seconds was reached”


Error in download.file(url, destfile, method, mode = "wb", ...) : 
  cannot open URL 'https://cran.r-project.org/src/contrib/rliger_2.2.1.tar.gz'


Warning message in download.packages(pkgs, destdir = tmpd, available = available, :
“download of package ‘rliger’ failed”
Warning message in install.packages("rliger"):
“installation of package ‘hdf5r.Extra’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



## Define Activity Programs

In [ ]:
library(rlist)

ERROR: Error in library(rliger): there is no package called ‘rliger’


In [7]:
library(rliger)
library(RANN)

breast.data <- readRDS('Data/breast.data.rds')
NMF_results <- iNMF_ksweep(breast.data, k.max = 40)

ERROR: Error in library(rliger): there is no package called ‘rliger’
